## Создание выборки из матриц отношений

In [ ]:
import numpy as np
import re
from time import sleep
import chess 
import chess.pgn
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
class Cell: 
    def __init__(self, x, y, t, c):
        self.x = x 
        self.y = y 
        self.type = t
        self.color = c
        self.attack = [] 
        self.attacked = []
        
def move_by_black_pawn(pawn, cells):
    if (pawn.x + 1 < 8) and (cells[pawn.x + 1][pawn.y].color == 0):
        pawn.attack.append((cells[pawn.x + 1][pawn.y].type, pawn.x + 1, pawn.y)) 
        cells[pawn.x + 1][pawn.y].attacked.append(pawn)
    if (pawn.x == 1) and (pawn.x + 2 < 8) and (cells[pawn.x + 2][pawn.y].color == 0):
        pawn.attack.append((cells[pawn.x + 2][pawn.y].type, pawn.x + 1, pawn.y)) 
        cells[pawn.x + 2][pawn.y].attacked.append(pawn)
        
def move_by_white_pawn(pawn, cells):
    if (pawn.x - 1 >= 0) and (cells[pawn.x - 1][pawn.y].color == 0):
        pawn.attack.append((cells[pawn.x + 1][pawn.y].type, pawn.x + 1, pawn.y)) 
        cells[pawn.x + 1][pawn.y].attacked.append(pawn)
    if (pawn.x == 1 and pawn.x - 2 >= 0) and (cells[pawn.x - 2][pawn.y].color == 0):
        pawn.attack.append((cells[pawn.x + 2][pawn.y].type, pawn.x + 1, pawn.y)) 
        cells[pawn.x + 2][pawn.y].attacked.append(pawn)    
        
def attack_by_black_pawn(pawn, cells):
    if (pawn.x + 1 < 8) and (pawn.y - 1 >= 0) and (pawn.color != cells[pawn.x + 1][pawn.y - 1].color) and (cells[pawn.x + 1][pawn.y - 1].color != 0): 
        pawn.attack.append((cells[pawn.x + 1][pawn.y - 1].type, pawn.x + 1, pawn.y - 1)) 
        cells[pawn.x + 1][pawn.y - 1].attacked.append(pawn) 
        
    if (pawn.x + 1 < 8) and (pawn.y + 1 < 8) and (pawn.color != cells[pawn.x + 1][pawn.y + 1].color) and (cells[pawn.x + 1][pawn.y + 1].color != 0): 
        pawn.attack.append((cells[pawn.x + 1][pawn.y + 1].type, pawn.x + 1, pawn.y + 1)) 
        cells[pawn.x + 1][pawn.y + 1].attacked.append(pawn)
        
def attack_by_white_pawn(pawn, cells):
    if (pawn.x - 1 >= 0) and (pawn.y - 1 >= 0) and (pawn.color != cells[pawn.x - 1][pawn.y - 1].color) and (cells[pawn.x - 1][pawn.y - 1].color != 0): 
        pawn.attack.append((cells[pawn.x - 1][pawn.y - 1].type, pawn.x - 1, pawn.y - 1)) 
        cells[pawn.x - 1][pawn.y - 1].attacked.append(pawn) 
        
    if (pawn.x - 1 >= 0) and (pawn.y + 1 < 8) and (pawn.color != cells[pawn.x - 1][pawn.y + 1].color) and (cells[pawn.x - 1][pawn.y + 1].color != 0): 
        pawn.attack.append((cells[pawn.x - 1][pawn.y + 1].type, pawn.x - 1, pawn.y + 1)) 
        cells[pawn.x - 1][pawn.y + 1].attacked.append(pawn)

def attack_by_knight(knight, cells): 
    moves = [(-1, 2), 
             (-2, 1), 
             (-2, -1), 
             (-1, -2), 
             (1, -2), 
             (2, -1), 
             (2, 1), 
             (1, 2)]
    
    for move in moves: 
        if (knight.x + move[0] >= 0) and (knight.x + move[0] < 8) and (knight.y + move[1] >= 0) and (knight.y + move[1] < 8) and (knight.color != cells[knight.x + move[0]][knight.y + move[1]].color): 
            knight.attack.append((knight.x + move[0], knight.y + move[1])) 
            cells[knight.x + move[0]][knight.y + move[1]].attacked.append(knight)
            
def attack_by_bishop(bishop, cells): 
    move_x = bishop.x + 1 
    move_y = bishop.y + 1 
    
    while (move_x < 8) and (move_y < 8): 
        if (bishop.color != cells[move_x][move_y].color):
            bishop.attack.append((move_x, move_y)) 
            cells[move_x][move_y].attacked.append(bishop) 
        
        if cells[move_x][move_y].type != 0: 
            break 
            
        move_x += 1 
        move_y += 1

    move_x = bishop.x + 1
    move_y = bishop.y - 1
    while (move_x < 8) and (move_y >= 0):
        if (bishop.color != cells[move_x][move_y].color):
            bishop.attack.append((move_x, move_y))
            cells[move_x][move_y].attacked.append(bishop)
        
        if cells[move_x][move_y].type != 0:
            break
            
        move_x += 1
        move_y -= 1

    move_x = bishop.x - 1
    move_y = bishop.y - 1
    
    while (move_x >= 0) and (move_y >= 0):
        if (bishop.color != cells[move_x][move_y].color):
            bishop.attack.append((move_x, move_y))
            cells[move_x][move_y].attacked.append(bishop)
        
        if cells[move_x][move_y].type != 0:
            break
            
        move_x -= 1
        move_y -= 1

    move_x = bishop.x - 1
    move_y = bishop.y + 1
    
    while (move_x >= 0) and (move_y < 8):
        if (bishop.color != cells[move_x][move_y].color):
            bishop.attack.append((move_x, move_y))
            cells[move_x][move_y].attacked.append(bishop)
        
        if cells[move_x][move_y].type != 0:
            break
            
        move_x -= 1
        move_y += 1

        
def attack_by_rook(rook, cells): 
    move_x = rook.x + 1 
    
    while move_x < 8:
        if (rook.color != cells[move_x][rook.y].color):
            rook.attack.append((move_x, rook.y)) 
            cells[move_x][rook.y].attacked.append(rook) 
        
        if cells[move_x][rook.y].type != 0: 
            break 
        move_x += 1

    move_x = rook.x - 1
    
    while move_x >= 0:
        if (rook.color != cells[move_x][rook.y].color):
            rook.attack.append((move_x, rook.y))
            cells[move_x][rook.y].attacked.append(rook)
            
        if cells[move_x][rook.y].type != 0:
            break
        move_x -= 1

    move_y = rook.y + 1
    
    while move_y < 8:
        if (rook.color != cells[rook.x][move_y].color):
            rook.attack.append((rook.x, move_y))
            cells[rook.x][move_y].attacked.append(rook)
            
        if cells[rook.x][move_y].type != 0:
            break
        move_y += 1

    move_y = rook.y - 1
    
    while move_y >= 0:
        if (rook.color != cells[rook.x][move_y].color):
            rook.attack.append((rook.x, move_y))
            cells[rook.x][move_y].attacked.append(rook)
            
        if cells[rook.x][move_y].type != 0:
            break
        move_y -= 1

        
def attack_by_queen(queen, cells): 
    attack_by_rook(queen, cells) 
    attack_by_bishop(queen, cells)
    
    

def attack_by_king(king, cells): 
    moves = [(-1, 0), (-1, 1), (-1, -1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1)] 
    
    for move in moves: 
        if (king.x + move[0] >= 0) and (king.x + move[0] < 8) and (king.y + move[1] >= 0) and (king.y + move[1] < 8) and (king.color != cells[king.x + move[0]][king.y + move[1]].color): 
            king.attack.append((king.x + move[0], king.y + move[1])) 
            cells[king.x + move[0]][king.y + move[1]].attacked.append(king)
            

In [ ]:
def make_binary_matrix(board): 
    figures = [] 
    cells = [[] for _ in range(8)] 

    for i in range(8): 
        for j in range(8): 
            if board[i][j] == 0:
                cells[i].append(Cell(i, j, board[i][j], 0))
            elif board[i][j].islower():
                figures.append(Cell(i, j, board[i][j], -1))
                cells[i].append(Cell(i, j, board[i][j], -1))
            elif board[i][j].isupper():
                figures.append(Cell(i, j, board[i][j], 1))
                cells[i].append(Cell(i, j, board[i][j], 1))


    for figure in figures: 
        if figure.type == 'p': 
            attack_by_black_pawn(figure, cells)
            move_by_black_pawn(figure, cells)
        elif figure.type == 'P': 
            attack_by_white_pawn(figure, cells)
            move_by_white_pawn(figure, cells)
        elif figure.type == 'n' or figure.type == 'N': 
            attack_by_knight(figure, cells)
        elif figure.type == 'b' or figure.type == 'B': 
            attack_by_bishop(figure, cells) 
        elif figure.type == 'r' or figure.type == 'R': 
            attack_by_rook(figure, cells) 
        elif figure.type == 'q' or figure.type == 'Q': 
            attack_by_queen(figure, cells) 
        elif figure.type == 'k' or figure.type == 'K': 
            attack_by_king(figure, cells)

    matrix_binary = [[0] * 64 for _ in range(len(figures))]

    for i in range(len(figures)): 
        for attacked in figures[i].attack: 
            matrix_binary[i][attacked[0] * 8 + attacked[1]] = 1
    return matrix_binary

In [ ]:
def board_to_array(board):
    # Создаем пустой двумерный массив размером 8x8
    board_array = [[None for j in range(8)] for i in range(8)]

    # Проходим по каждой позиции на шахматной доске
    for row in range(8):
        for col in range(8):
            # Получаем фигуру с позиции и преобразуем её в символ
            piece = board.piece_at(8 * row + col)
            symbol = piece.symbol() if piece else 0

            # Сохраняем символ фигуры в двумерный массив
            board_array[row][col] = symbol

    return board_array

In [ ]:
def read_and_write_to_file(input_file_path, output_file_bin_path):
    with open(input_file_path) as pgn_file, open(output_file_bin_path, "w") as output_bin_file:
        while True:
            game = chess.pgn.read_game(pgn_file)
            board = chess.Board()

            for number, move in enumerate(game.mainline_moves()): 
                board.push(move)
                board_array = board_to_array(board)
                binary_matrix = make_binary_matrix(board_array)

                for row in binary_matrix:
                    # Проходимся по каждому элементу в ряду
                    for item in row:
                        # Записываем элемент в файл
                        output_bin_file.write(str(item) + " ")
                    # Переход на новую строку между рядами
                    output_bin_file.write("\n")      